## 1 Импорт и конфигурация

In [16]:
import os
import pathlib
import nltk
import numpy as np
import pandas as pd

from copy import deepcopy
from tqdm import tqdm
from collections import Counter

In [17]:
def random_keyboard(init_keyboard, permutable_keys):
    random_keyboard = deepcopy(init_keyboard)
    permutable_keys_copy = deepcopy(permutable_keys)

    for i in range(len(random_keyboard)):
        for j in range(len(random_keyboard[i])):
            if init_keyboard[i][j] is None:
                random_key = np.random.choice(permutable_keys_copy)
                permutable_keys_copy.remove(random_key)
                random_keyboard[i][j] = random_key

    return random_keyboard

In [34]:
RU_BASE_PART_DIR = '/kaggle/input/langs-base-parts/ru/'
EN_BASE_PART_DIR = '/kaggle/input/langs-base-parts/en/'

RU_SENTENCE_CLASSES = (
    'prn_preposadj_v',
    's_instr_v',
    's_v',
    'adj_noun',
    'adv_adv',
    'adv_verb',
    'business',
    'facts',
    'questions',
    'poems',
    'prose',
    'publicism',
)

EN_SENTENCE_CLASSES = (
    'bnccorpus',
    'twitterconvcorpus',
    'dialogues'
)

RU_COUNTED_KEYS = [chr(i) for i in range(1072, 1104)] + ['ё'] + ['space', 'enter'] + ['.', ',']
RU_PERMUTABLE_KEYS = [chr(i) for i in range(1072, 1104)] + ['ё'] + ['.', ',']

EN_COUNTED_KEYS = [chr(i) for i in range(97, 123)] + ['space', 'enter'] + ['.', ',', '!', '?', '-', ':', ';', '(', ')']
EN_PERMUTABLE_KEYS = [chr(i) for i in range(97, 123)] + ['.', ',', '!', '?', '-', ':', ';', '(', ')']

INIT_HEX_KEYBOARD = [
    ['inv', 'inv', 'lang', None, None, None, None, '?123', 'inv'],
    ['inv', 'settings', None, None, None, None, None, 'backspace', 'inv'],
    ['inv', 'inv', None, None, None, None, None, None, 'inv'],
    ['inv', None, None, None, 'space', None, None, 'enter', 'inv'],
    ['inv', 'inv', None, None, None, None, None, None, 'inv'],
    ['inv', 'move', None, None, None, None, None, 'capslock', 'inv'],
    ['inv', 'inv', 'exit', None, None, None, None, 'shift', 'inv'],
]

A_S = 1
A_H = 0.537634
B_H = 0.930605

In [35]:
ru_random_keyboard = random_keyboard(INIT_HEX_KEYBOARD, RU_PERMUTABLE_KEYS)
ru_random_keyboard

[['inv', 'inv', 'lang', 'н', 'к', 'щ', ',', '?123', 'inv'],
 ['inv', 'settings', 'д', 'а', 'ъ', 'ф', 'р', 'backspace', 'inv'],
 ['inv', 'inv', 'х', 'й', 'е', '.', 'ц', 'л', 'inv'],
 ['inv', 'б', 'ш', 'п', 'space', 'м', 'у', 'enter', 'inv'],
 ['inv', 'inv', 'з', 'ч', 'ё', 'ь', 'т', 'э', 'inv'],
 ['inv', 'move', 'в', 'г', 'ж', 'ы', 'с', 'capslock', 'inv'],
 ['inv', 'inv', 'exit', 'я', 'ю', 'и', 'о', 'shift', 'inv']]

In [36]:
en_random_keyboard = random_keyboard(INIT_HEX_KEYBOARD, EN_PERMUTABLE_KEYS)
en_random_keyboard

[['inv', 'inv', 'lang', 'n', 'v', 'z', 'i', '?123', 'inv'],
 ['inv', 'settings', '-', 'k', 'c', 's', '?', 'backspace', 'inv'],
 ['inv', 'inv', 'y', 'q', ';', 'e', 'h', 'm', 'inv'],
 ['inv', '!', 'g', 'l', 'space', ')', 'r', 'enter', 'inv'],
 ['inv', 'inv', '(', 'o', 'u', 't', 'x', 'w', 'inv'],
 ['inv', 'move', 'b', '.', 'j', 'a', 'f', 'capslock', 'inv'],
 ['inv', 'inv', 'exit', 'd', ',', 'p', ':', 'shift', 'inv']]

## 2 Чтение данных

In [37]:
print(f'Количество оптимизируемых символов в русской раскладке: {len(RU_COUNTED_KEYS)}')
print(f'Количество оптимизируемых символов в английской раскладке: {len(EN_COUNTED_KEYS)}')

Количество оптимизируемых символов в русской раскладке: 37
Количество оптимизируемых символов в английской раскладке: 37


In [38]:
def read_lang_base_part(base_part_dir, sentence_classes, cur_pattern='*.txt'):
    cur_dir = pathlib.Path(base_part_dir)
    df = pd.DataFrame()
    
    for file in cur_dir.glob(cur_pattern): 
        print(f'Обработка {file}')
        
        cur_df = pd.read_csv(file, sep='\n', header=None, names=['text'], index_col=0)
    
        for sentence_class in sentence_classes:
            if sentence_class in str(file).split('/')[-1].lower():
                cur_df['class'] = sentence_class
        df = pd.concat((df, cur_df))
    
    return df.reset_index()

In [97]:
df_ru = read_lang_base_part(RU_BASE_PART_DIR, RU_SENTENCE_CLASSES)
df_en = read_lang_base_part(EN_BASE_PART_DIR, EN_SENTENCE_CLASSES)

Обработка /kaggle/input/langs-base-parts/ru/facts4_2s.txt
Обработка /kaggle/input/langs-base-parts/ru/facts6_2s.txt
Обработка /kaggle/input/langs-base-parts/ru/adv_adv.plain.txt
Обработка /kaggle/input/langs-base-parts/ru/S_V_INSTR.plain.txt
Обработка /kaggle/input/langs-base-parts/ru/PRN_PreposAdj_V.plain.txt
Обработка /kaggle/input/langs-base-parts/ru/questions6.txt
Обработка /kaggle/input/langs-base-parts/ru/facts7_2s.txt
Обработка /kaggle/input/langs-base-parts/ru/poems_Pushkin.txt
Обработка /kaggle/input/langs-base-parts/ru/publicism_Tolstoy.txt
Обработка /kaggle/input/langs-base-parts/ru/S_V_INF.plain.txt
Обработка /kaggle/input/langs-base-parts/ru/business.txt
Обработка /kaggle/input/langs-base-parts/ru/facts7_1s.txt
Обработка /kaggle/input/langs-base-parts/ru/prose_Chekhov.txt
Обработка /kaggle/input/langs-base-parts/ru/adj_noun.plain.txt
Обработка /kaggle/input/langs-base-parts/ru/facts4.txt
Обработка /kaggle/input/langs-base-parts/ru/prose_Lermontove.txt
Обработка /kaggle/inp

In [98]:
print('----- Русский язык -----', end='\n\n')

print(f'Размерность датасета: {df_ru.shape}', end='\n\n')
print(f'Распределение классов:\n{df_ru["class"].value_counts(normalize=True).mul(100).round(2).astype(str) + "%"}', end='\n\n')
print(f'5 случайных элементов:\n{df_ru.sample(5)}')

----- Русский язык -----

Размерность датасета: (2702849, 2)

Распределение классов:
s_v                49.02%
adj_noun           21.66%
facts              19.46%
questions           4.96%
prn_preposadj_v     3.04%
poems               0.77%
adv_verb            0.75%
prose               0.22%
adv_adv             0.06%
business            0.04%
s_instr_v           0.02%
publicism           0.01%
Name: class, dtype: object

5 случайных элементов:
                                   text     class
1815839             Милена согласилась.       s_v
2633909  внутри догорают дрова в камине     facts
1418756                    Новом Яндекс  adj_noun
1256592                   ОБЩИЕ РЕСУРСЫ  adj_noun
1357495              Черниговскую улицу  adj_noun


In [99]:
print('----- Английский язык -----', end='\n\n')

print(f'Размерность датасета: {df_en.shape}', end='\n\n')
print(f'Распределение классов:\n{df_en["class"].value_counts(normalize=True).mul(100).round(2).astype(str) + "%"}', end='\n\n')
print(f'5 случайных элементов:\n{df_en.sample(5)}')

----- Английский язык -----

Размерность датасета: (620819, 2)

Распределение классов:
bnccorpus            97.69%
twitterconvcorpus     1.71%
dialogues              0.6%
Name: class, dtype: object

5 случайных элементов:
                                          text      class
451596                                    Yeah  bnccorpus
354574                                      Mm  bnccorpus
292397            Yeah but one's not gonna fit  bnccorpus
317605                                      Mm  bnccorpus
132312  You're not an offensive person are you  bnccorpus


# 2 Анализ биграмм

In [100]:
def tokenize_by_letters(df):
    tokenized_text = []
    
    for sentence in tqdm(df['text']):
        for letter in str(sentence).lower():
            tokenized_text.append(letter)
    return tokenized_text

In [101]:
def get_bigram_probs(tokenized_text):
    bigrams = nltk.bigrams(tokenized_text)
    freq_dists = nltk.FreqDist(bigrams)
    prob_dists = nltk.MLEProbDist(freq_dists)
    
    bigram_probs = []
    for sample in tqdm(freq_dists.keys(),  position=0, leave=True):
        bigram_probs.append((sample, prob_dists.prob(sample)))
    
    return bigram_probs

In [102]:
ru_tokenized_text = tokenize_by_letters(df_ru)
en_tokenized_text = tokenize_by_letters(df_en)

print(f'Количество символов в полном русском тексте: {len(ru_tokenized_text)}')
print(f'Количество символов в полном английском тексте: {len(en_tokenized_text)}')

100%|██████████| 620819/620819 [00:02<00:00, 240214.33it/s]

Количество символов в полном русском тексте: 61475676
Количество символов в полном английском тексте: 19291457


In [103]:
ru_letter_counts = dict(Counter(ru_tokenized_text))
en_letter_counts = dict(Counter(en_tokenized_text))

print(f'Количество уникальных символов в русском языке: {len(ru_letter_counts)}')
print(f'10 самых частых символов русского языка:\n{sorted(ru_letter_counts.items(), key=lambda item: item[1], reverse=True)[:10]}', end='\n\n')

print(f'Количество уникальных символов в английском языке: {len(en_letter_counts)}')
print(f'10 самых частых символов английского языка языка:\n{sorted(en_letter_counts.items(), key=lambda item: item[1], reverse=True)[:10]}')

Количество уникальных символов в русском языке: 141
10 самых частых символов русского языка:
[(' ', 6335435), ('о', 5267409), ('а', 4658398), ('е', 4035798), ('и', 3512394), ('т', 3510625), ('с', 3279792), ('н', 3155055), ('л', 3082723), ('р', 2767487)]

Количество уникальных символов в английском языке: 410
10 самых частых символов английского языка языка:
[(' ', 3499240), ('e', 1811313), ('t', 1571776), ('o', 1370323), ('a', 1191684), ('h', 1068078), ('i', 1052504), ('n', 1036466), ('s', 868650), ('r', 705838)]


In [104]:
ru_bigram_probs = get_bigram_probs(ru_tokenized_text)
en_bigram_probs = get_bigram_probs(en_tokenized_text)

print(f'Количество уникальных биграмм в русском языке: {len(ru_bigram_probs)}')
print(f'10 самых частых биграмм в русском языке:\n{sorted(ru_bigram_probs, key=lambda item: item[1], reverse=True)[:10]}', end='\n\n')

print(f'Количество уникальных биграмм в английском языке: {len(en_bigram_probs)}')
print(f'10 самых частых биграмм в английском языке:\n{sorted(en_bigram_probs, key=lambda item: item[1], reverse=True)[:10]}')

100%|██████████| 4426/4426 [00:00<00:00, 598182.30it/s]

Количество уникальных биграмм в русском языке: 4096
10 самых частых биграмм в русском языке:
[((' ', 'п'), 0.016663176126166974), (('я', ' '), 0.012559731959022817), (('а', 'л'), 0.012123234108450212), ((' ', 'с'), 0.01176003354172199), (('с', 'т'), 0.011338728692283574), (('т', 'ь'), 0.011263951148157382), (('п', 'о'), 0.01023260013005144), (('а', ' '), 0.010117839291719855), (('н', 'а'), 0.009711808776398144), ((' ', 'в'), 0.009356513775570582)]

Количество уникальных биграмм в английском языке: 4426
10 самых частых биграмм в английском языке:
[(('e', ' '), 0.030721216687843573), ((' ', 't'), 0.02671172149992204), (('t', ' '), 0.026292053850160403), (('t', 'h'), 0.024743181644765435), (('h', 'e'), 0.021388484104051038), (('s', ' '), 0.01738697172468475), ((' ', 'i'), 0.015997548344717994), ((' ', 'a'), 0.01527106092977119), (('o', 'u'), 0.01423645783915947), (('d', ' '), 0.013487784436799379)]


In [105]:
def filter_bigram_probs(bigram_probs, counted_keys):
    filtered_bigram_probs = []
    
    for bigram in tqdm(bigram_probs):
        new_bigram = [bigram[0][0], bigram[0][1]]
        
        if bigram[0][0] == ' ':
            new_bigram[0] = 'space'
        if bigram[0][1] == ' ':
            new_bigram[1] = 'space'
        if bigram[0][0] == '\n':
            new_bigram[0] = 'enter'
        if bigram[0][1] == '\n':
            new_bigram[1] = 'enter'
        if new_bigram[0] in counted_keys and new_bigram[1] in counted_keys:
            filtered_bigram_probs.append(((new_bigram[0], new_bigram[1]), bigram[1]))
            
    return filtered_bigram_probs

In [116]:
ru_filtered_bigram_probs = filter_bigram_probs(ru_bigram_probs, RU_COUNTED_KEYS)

100%|██████████| 4096/4096 [00:00<00:00, 408121.37it/s]


In [117]:
print(f'Количество уникальных отфильтрованных биграмм: {len(ru_filtered_bigram_probs)}')
print(f'10 самых частых отфильтрованных биграмм:')
sorted(ru_filtered_bigram_probs, key=lambda item: item[1], reverse=True)[:10]

Количество уникальных отфильтрованных биграмм: 1255
10 самых частых отфильтрованных биграмм:


[(('space', 'п'), 0.016663176126166974),
 (('я', 'space'), 0.012559731959022817),
 (('а', 'л'), 0.012123234108450212),
 (('space', 'с'), 0.01176003354172199),
 (('с', 'т'), 0.011338728692283574),
 (('т', 'ь'), 0.011263951148157382),
 (('п', 'о'), 0.01023260013005144),
 (('а', 'space'), 0.010117839291719855),
 (('н', 'а'), 0.009711808776398144),
 (('space', 'в'), 0.009356513775570582)]

In [118]:
en_filtered_bigram_probs = filter_bigram_probs(en_bigram_probs, EN_COUNTED_KEYS)

100%|██████████| 4426/4426 [00:00<00:00, 423632.27it/s]


In [119]:
print(f'Количество уникальных отфильтрованных биграмм: {len(en_filtered_bigram_probs)}')
print(f'10 самых частых отфильтрованных биграмм:')
sorted(en_filtered_bigram_probs, key=lambda item: item[1], reverse=True)[:10]

Количество уникальных отфильтрованных биграмм: 1133
10 самых частых отфильтрованных биграмм:


[(('e', 'space'), 0.030721216687843573),
 (('space', 't'), 0.02671172149992204),
 (('t', 'space'), 0.026292053850160403),
 (('t', 'h'), 0.024743181644765435),
 (('h', 'e'), 0.021388484104051038),
 (('s', 'space'), 0.01738697172468475),
 (('space', 'i'), 0.015997548344717994),
 (('space', 'a'), 0.01527106092977119),
 (('o', 'u'), 0.01423645783915947),
 (('d', 'space'), 0.013487784436799379)]

In [80]:
def get_bigram_probs_vec(bigram_probs):
    if bigram_probs is not None:
        return np.array([elem[1] for elem in bigram_probs])

In [92]:
ru_bigram_probs_vec = get_bigram_probs_vec(ru_filtered_bigram_probs)
en_bigram_probs_vec = get_bigram_probs_vec(en_filtered_bigram_probs)

print(f'10 самых больших вероятностей биграмм в русском языке (для сравнения):\n{sorted(ru_bigram_probs_vec, reverse=True)[:10]}', end='\n\n')
print(f'10 самых больших вероятностей биграмм в английском языке (для сравнения):\n{sorted(en_bigram_probs_vec, reverse=True)[:10]}')

10 самых больших вероятностей биграмм в русском языке (для сравнения):
[0.012123234108450212, 0.011338728692283574, 0.011263951148157382, 0.01023260013005144, 0.009711808776398144, 0.009086716005964961, 0.009060543052190968, 0.008426113255364825, 0.008118154050362196, 0.008051770070031765]

10 самых больших вероятностей биграмм в английском языке (для сравнения):
[0.024743181644765435, 0.021388484104051038, 0.01423645783915947, 0.012596198026732664, 0.012080373819373717, 0.011880959114750074, 0.011472747313629413, 0.009970009521313477, 0.009804184816324906, 0.009647275975437002]


In [84]:
def square_dist(x1, x2, a=1):
    return a * np.linalg.norm(np.array(x2) - np.array(x1))

In [85]:
def hex_dist(x1, x2, a=1, b=1):
    hdist = 2 * (x1[1] - x2[1]) - (x1[0] % 2 - x2[0] % 2)
    vdist = x1[0] - x2[0]
    return ((a * hdist)**2 + (b * vdist)**2) ** (1/2)

In [86]:
def key_index(key, keyboard_matrix):
    indexes = []
    
    for i in range(len(keyboard_matrix)):
        for j in range(len(keyboard_matrix[i])):
            
            if type(keyboard_matrix[i][j]) != list:
                if keyboard_matrix[i][j] == key:
                    indexes.append((i, j))
            else:
                for k in keyboard_matrix[i][j]:
                    if k == key:
                        indexes.append((i, j))
    return np.array(indexes)

In [87]:
def get_dists_vec(bigram_probs, keyboard_matrix, dist_func='square', a_s=1, a_h=1, b_h=1):
    assert dist_func in ('square', 'hex')
    
    result_vec = []
    
    for bigram in bigram_probs:
        key_indexes_1 = key_index(bigram[0][0], keyboard_matrix)
        key_indexes_2 = key_index(bigram[0][1], keyboard_matrix)

        dists = []
        for key_index_1 in key_indexes_1:
            for key_index_2 in key_indexes_2:
                if dist_func == 'square':
                    dists.append(square_dist(key_index_1, key_index_2, a=a_s))
                elif dist_func == 'hex':
                    dists.append(hex_dist(key_index_1, key_index_2, a=a_h, b=b_h))
                
        result_vec.append(min(dists))
            
    return np.array(result_vec)

In [93]:
ru_dists_vec = get_dists_vec(ru_filtered_bigram_probs, ru_random_keyboard, dist_func='hex', a_h=A_H, b_h=B_H)
en_dists_vec = get_dists_vec(en_filtered_bigram_probs, en_random_keyboard, dist_func='hex', a_h=A_H, b_h=B_H)

print(f'10 самых близких расстояний в русском языке (для сравнения):\n{sorted(ru_dists_vec, reverse=True)[:10]}', end='\n\n')
print(f'10 самых близких расстояний в анлглийском языке (для сравнения):\n{sorted(en_dists_vec, reverse=True)[:10]}')

10 самых близких расстояний в русском языке (для сравнения):
[7.050924010410904, 7.050924010410904, 7.050924010410904, 7.050924010410904, 6.7129514675037685, 6.7129514675037685, 6.7129514675037685, 6.7129514675037685, 6.539825644992456, 6.539825644992456]

10 самых близких расстояний в анлглийском языке (для сравнения):
[7.050924010410904, 7.050924010410904, 7.050924010410904, 7.050924010410904, 6.7129514675037685, 6.7129514675037685, 6.7129514675037685, 6.539825644992456, 6.539825644992456, 6.539825644992456]
